In [ ]:
from pathlib import Path
import numpy as np
import importlib
from matplotlib import pyplot as plt
import humanize
import sys
import os, psutil
import time
import shutil
import pandas as pd
import seaborn as sns
import warnings
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import logging
import string
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.ticker import AutoLocator

import astrocast.reduction as red
import astrocast.clustering as clust
import astrocast.analysis as ana
import astrocast.autoencoders as AE
import astrocast.helper as helper
import astrocast.experiments as exp

for pack in [red, clust, helper, ana, AE, helper, exp]:
    importlib.reload(pack)


In [ ]:
file = Path("/media/janrei1/data/data_paper/22A8x5/22A8x5-2.h5")
!astrocast visualize-h5 {file}

## Load data

In [ ]:
dn = ana.Video(file, loc="dn/ast", lazy=False)
dn = dn.get_data()

In [ ]:
df = ana.Video(file, loc="df/ast", lazy=False)
df = df.get_data()

In [ ]:
df_norm = df / df.max()
dn_norm = dn / dn.max()

## visualize

In [ ]:
n_frame = 200
row_titles = {'A': 'before subtraction', 'B': 'after subtraction'}
z0, z1 = 400, 800

pixels = [(144, 107), (98, 67), (111, 223)]
colors = sns.color_palette('husl', n_colors=len(pixels))

sns.set_style("white")

fig, axx = plt.subplots(2, 2, figsize=(8, 6),
                        gridspec_kw={'width_ratios': [1, 2]})

ax0, ax1 = axx[0, 0], axx[1, 0]
ax0.imshow(dn[n_frame, :, :])
ax1.imshow(df[n_frame, :, :])

for ax in (ax0, ax1):
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    for i, (px, py) in enumerate(pixels):
        ax.scatter(py, px, marker='<', edgecolor='white', color=colors[i], s=50, alpha=0.5)

ax0, ax1 = axx[0, 1], axx[1, 1]
for i, (px, py) in enumerate(pixels):
    
    trace1 = dn[z0:z1, px, py].copy().astype(float)
    ax0.plot(trace1, color=colors[i], alpha=0.9)
    
    trace2 = df[z0:z1, px, py].copy().astype(float)
    ax1.plot(trace2, color=colors[i], alpha=0.9)

for n, ax in enumerate(list(axx.flatten())):
    ax.text(-0.075, 1.05, string.ascii_uppercase[n], transform=ax.transAxes, size=14, weight='bold')

axx[0, 0].set_ylabel("before subtraction")
axx[1, 0].set_ylabel("after subtraction")

axx[0, 1].set_ylabel("pixel intensity")
axx[1, 1].set_ylabel("pixel intensity")

axx[0, 1].set_xlabel("frames")
axx[1, 1].set_xlabel("frames")

sns.despine(top=True, right=True, ax=axx[0, 1])
sns.despine(top=True, right=True, ax=axx[1, 1])

plt.tight_layout()

fig_name = "4"
save_path = Path.cwd().parent.joinpath(f"{fig_name}.png")
fig.savefig(save_path, dpi=(260))

legend = """
Comparison before and after the application of background subtraction to fluorescence imaging data. (A, C) fluorescence images before and after background subtraction, respectively, illustrating the removal of extraneous noise. After subtraction, the background is close to zero and exhibits overall lower noise levels. (B, D) Intensity traces corresponding to the marked points on the images are plotted over time (400 frames, 50s). These traces highlight the efficacy of background subtraction, with the post-subtraction traces approaching zero on the y-axis, thereby indicating a substantial reduction of background, while signal amplitude is conserved. However, the signal shapes, especially in the red trace, exhibit slight alterations post-subtraction. Moreover, the green trace reveals the inadvertent introduction of spurious events, serving as a cautionary example of how background subtraction can inadvertently affect data integrity if parameters are not judiciously optimized.
"""
legend_path = Path.cwd().parent.joinpath(f"{fig_name}.txt")
with open(legend_path, 'w') as f:
    f.write(legend)

In [ ]:




fig, axx = plt.subplots(len(pixels) + 1, 2,
                        figsize=(11, 11),
                        gridspec_kw={'height_ratios': [len(pixels)] + len(pixels) * [1]})

data_frame = data_norm[n_frame, :, :]
dn_frame = dn_norm[n_frame, :, :]

for i, (idx_col, frame) in enumerate([(0, data_frame), (1, dn_frame)]):
    
    ax = axx[0, idx_col]
    ax.imshow(frame)
    
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    for i, (px, py) in enumerate(pixels):
        ax.scatter(py, px, marker='<', edgecolor='white', color=colors[i], s=50, alpha=0.5)

for i, (px, py) in enumerate(pixels):
    
    ax0, ax1 = axx[i + 1, 0], axx[i + 1, 1]
    ax0.sharey(ax1)
    
    trace1 = data_norm[z0:z1, px, py].copy().astype(float)
    trace1 -= trace1.min()
    div = trace1.max()
    trace1 /= div
    
    trace2 = dn_norm[z0:z1, px, py].copy()
    trace2_zero = trace2[0]
    trace2_min = trace2.min()
    trace2_max = trace2.max()
    
    trace2 -= trace2_min
    trace2 /= div
    
    ax0.plot(trace1, color=colors[i], alpha=0.9)
    ax1.plot(trace2, color=colors[i], alpha=0.9)

axx[0, 0].set_title("before denoising")
axx[0, 1].set_title("after denoising")

axx[1, 0].set_title("before denoising")
axx[1, 1].set_title("after denoising")

for ax in axx[1:, :].flatten():
    
    ax.xaxis.set_major_locator(AutoLocator())  # solution
    ax.xaxis.set_tick_params(labelbottom=True)
    
    sns.despine(top=True, right=True, ax=ax)

for ax in axx[1:-1, :].flatten():
    ax.set_xticklabels([])

# for ax in axx[1:, 1].flatten():
#     ax.set_yticklabels([])

for ax in axx[1:, 0].flatten():
    ax.set_ylabel("normalized\nintensity")

for ax in axx[-1, :].flatten():
    ax.set_xlabel("Frames")

for n, ax in enumerate(list(axx.flatten())):
    ax.text(-0.075, 1.05, string.ascii_uppercase[n], transform=ax.transAxes, size=14, weight='bold')

plt.tight_layout()

save_path = Path.cwd().parent.joinpath("3.png")
print(save_path)
fig.savefig(save_path, dpi=(260))

legend = """
Effect of denoising on astrocytic calcium imaging data. (A) A single 256x256 pixel frame prior to denoising, where background noise is evident. (B) The same frame following denoising with enhanced clarity. (C, E, G, and I) Pixel intensity over 400 frames (50s) before the application of the denoising algorithm, showcasing the original signal variation, after normalization. (D, F, H, J) Corresponding traces after denoising, illustrating a stabilized intensity profile, with preserved signal characteristics. The same normalization parameters were applied to both original and denoised traces to ensure comparability of noise levels. The denoising algorithm was executed on a (128, 128) field of view, incorporating a context of five adjacent frames for each target frame, with no gap frames. Parameters for the denoising model included a training period of 50 epochs, a learning rate of 0.0001, momentum of 0.9, and a stack of three layers with 64 kernels of size three in the initial layer, omitting batch normalization. During inference, a strategy of a 10-pixel overlap in all directions, complemented by 'edge' padding, was employed. 
"""
legend_path = Path.cwd().parent.joinpath("3.txt")
with open(legend_path, 'w') as f:
    f.write(legend)